# Topic Modeling on News Snippets Using LDA
**Author:** Virginia Herrero

## Import Libraries and Download Resources

Import essential libraries for text preprocessing, topic modeling, and download required NLTK resources.

In [1]:
# Text processing
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer
from gensim.utils import simple_preprocess

# Download NLTK resources
nltk.download("stopwords")
nltk.download("wordnet")

# Topic Modeling
import gensim
import gensim.corpora as corpora
from gensim.models import LdaMulticore, CoherenceModel

# Utilities
from pprint import pprint

[nltk_data] Downloading package stopwords to C:\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to C:\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


## Define the Corpus

In natural language processing, a corpus is a collection of written or spoken texts that serves as the dataset for language-related tasks. The corpus is analyzed to identify language patterns and typically requires preprocessing and transformation into a format suitable for machine learning models.

In [2]:
corpus = [
    "The stock market closed higher today as tech shares rallied amid strong earnings reports.",
    "A major earthquake struck the coastal city early this morning, causing widespread damage.",
    "The government announced new policies aimed at reducing carbon emissions by 2030.",
    "Scientists discovered a new species of dinosaur in the remote mountains of Argentina.",
    "The local football team won the championship after a thrilling final match.",
    "Health officials urge citizens to get vaccinated as flu season approaches.",
    "A breakthrough in renewable energy technology promises cheaper solar panels.",
    "International leaders met to discuss trade agreements and economic cooperation.",
    "A popular film festival opened this weekend, showcasing independent movies from around the world.",
    "The city council approved plans for a new public park to promote green spaces."
]